In [1]:
import dropbox
import os
import pandas as pd
import numpy as np
import re
import sys
import datetime
import itertools
import io

In [2]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from utils.utils import *

In [4]:
db = dropbox_handler()
db.list_files('/Data/chicago-police-data/Import/input/TRRs/')

['TRR Data - 10.29 public.xlsx',
 'TRR Data - 10.29 lookup only.xlsx',
 'geocoded (with lat and lon in separate columns).xlsx',
 'column headers.xlsx',
 'column headers.pdf',
 'column headers.csv',
 '10655-FOIA-P046360-TRRdata_cleaned.xlsx']

### LOCAL PATH MUST BE SPECIFIED BY USER AS OUTPUT FILES ARE SAVED LOCALLY

In [ ]:
local_path = '/your/path/here/'

In [ ]:
path = '/Data/chicago-police-data/import'
db = dropbox_handler()
db.list_files(path+'/input')

In [4]:
location_code = db.download_file(path+'/doc/','Location_Code_Dictionary.csv')
location_code.dropna(how='all', inplace=True)
location_code['Location_Code']=location_code['Location_Code'].astype(int).astype(str)
location_code

/Data/Import/doc/Location_Code_Dictionary.csv


,Location_Code,Location_Value
0,1,Food Sales/Restaurant
1,2,Tavern/Liquor Store
2,3,Other Business Establishment
3,4,Police Building
4,5,Lockup Facility
5,6,Police Maintenance Facility
6,7,CPD Automotive Pound Facility
7,8,Other Police Property
8,9,Police Communications Systems
9,10,Court Room


### Reading Public File Locally

In [5]:
##file_loc = "/Users/thudson/Documents/Github/chicago-police-data with TRR Data/import/input/TRRs/"
file_loc = '/Data/Import/input/TRRs/'
file_name = 'TRR Data_10_29 public.xlsx'

In [6]:
sheets = db.download_file(file_loc,file_name,return_sheets=True)
##sheets = pd.ExcelFile(file_loc + file_name).sheet_names
sheets

/Data/Import/input/TRRs/TRR Data_10_29 public.xlsx


['Events',
 'Notes',
 'Officers',
 'Star #',
 'TRRs',
 'ActionsResponses',
 'WeaponDischarges',
 'Charges',
 'SubjectWeapons',
 'Statuses_OtherMembers',
 'IIColHeaders',
 'OfficerFrontPage',
 'SwornOfficers',
 'Awards',
 'SwornAll',
 'Settlements',
 'Units',
 'Lookups']

In [11]:
sheet = [sheet for sheet in sheets if sheet=='TRRs'][0]

final_df = pd.DataFrame()
metadata_df = pd.DataFrame()

##df = pd.read_excel(file_loc + file_name,sheetname=sheet)
df = db.download_file(file_loc,file_name,sheetname=sheet)

print(df.columns)

df.columns = ['TRR_Report_ID','RD_No','CR_Status','Event_No','Beat','District','Dist_Race',
              'Block','Direction','Street','Location','Date','Year','Month','Time_Mil','Indoor_Outdoor',
              'Lighting_Condition','Weather_Condition','Notify_OEMC','Notify_District_Sergeant',
              'Notify_OP_Command','Notify_Detective_Division','Number_Weapons_Discharged',
              'Party_Fired_First','Officer_Last_Name','Officer_First_Name','Officer_Gender',
              'Officer_Race','Star','Officer_Age','Appointed_Date','Unit_Assigned','Unit_Detail',
              'Assigned_Beat','Rank','Rank2','Duty_Status','Officer_Injured','Member_In_Uniform',
              'Subject_Gender','Subject_Race','Subject_Age','Subject_YOB','Subject_Armed','Subject_Injured',
              'Subject_Alleged_Injury','Compl%','Sworn_All_Check','Dup_Over','Unique_ID','WeapDischType',
              'GunHit','SwornAll2 Check','PO_TRRS_Event','POCounter','Booked_TRRs_Event','BookedCounter',
              'POHomCheck','HighestForceTRR']



## dropping created fields
df = df.drop(['CR_Status','District','Dist_Race','Year','Month','Star','Rank2','Compl%','Sworn_All_Check',
              'Dup_Over','Unique_ID','WeapDischType','GunHit','SwornAll2 Check','PO_TRRS_Event',
              'POCounter','Booked_TRRs_Event','BookedCounter','POHomCheck','HighestForceTRR'],axis=1)


final_df = final_df.append(df)
final_df.reset_index(drop=True,inplace=True)
metadata_df = metadata_df.append(metadata_dataset(df,sheet))
metadata_df.reset_index(drop=True,inplace=True)

/Data/Import/input/TRRs/TRR Data_10_29 public.xlsx
Index(['EVENT_NO', 'TRRs', 'Officers', 'Booked', 'CRID', 'Beat', 'District',
       'Date', 'Year', 'Quarter', 'Qyear', 'Time', 'TotalWeapDisc',
       'AnySubjArmed', 'NumSubjWeapons', 'SubjFirearms', 'SubjThreats',
       'SubjUnarmedAttack', 'SubjMeleeWeapon', 'SubjVehicle',
       'SubjOtherWeapon', 'Injury', 'InjuryAlleg', 'NumPOInj', 'NumPOOnDuty',
       'NumPOInUni', 'HighestRankOnScene', 'Most Resistance', 'Most Force',
       'ForceSocialVerbal', 'ForcePhysHold', 'ForcePhysStun',
       'ForcePhysDirect', 'ForceChem', 'ForceChemAuth', 'ForceTaser',
       'ForceImpactWep', 'ForceFirearm', 'ForceOther',
       'IncludesCoded'OtherForce'', 'DutyStatus', 'Uniform Status',
       'POFiredShots', 'POsGunshotHitPerson', 'TotalPOShotsFired',
       'TasersFired', 'ChemFired', 'OtherPOWep', 'PO_Black', 'PO_White',
       'PO_Hispanic', 'PO_Asian', 'PO_Native Am', 'POHomEvents'],
      dtype='object')


ValueError: Length mismatch: Expected axis has 54 elements, new values have 59 elements

In [ ]:
out_path = local_path+'/output/TRRs/'
saving_files = "TRRs"

final_df.to_csv(out_path+saving_files+".csv",index=False)
final_df.to_excel(out_path+saving_files+".xlsx",index=False)

metadata_df.to_csv(out_path+saving_files+"_metadata.csv",index=False)

In [ ]:
sheet = [sheet for sheet in sheets if sheet=='Officers'][0]

final_df = pd.DataFrame()
metadata_df = pd.DataFrame()

##df = pd.read_excel(file_loc + file_name,sheetname=sheet)
df = db.download_file(file_loc,file_name,sheetname=sheet)

df.columns = ['Officer_Last_Name','Officer_First_Name','Star']

final_df = final_df.append(df)
final_df.reset_index(drop=True,inplace=True)
metadata_df = metadata_df.append(metadata_dataset(df,sheet))
metadata_df.reset_index(drop=True,inplace=True)

In [ ]:
out_path = local_path+'/output/TRRs/'
saving_files = "Officers"

final_df.to_csv(out_path+saving_files+".csv",index=False)
final_df.to_excel(out_path+saving_files+".xlsx",index=False)

metadata_df.to_csv(out_path+saving_files+"_metadata.csv",index=False)

In [ ]:
sheet = [sheet for sheet in sheets if sheet=='Star #'][0]

final_df = pd.DataFrame()
metadata_df = pd.DataFrame()

##df = pd.read_excel(file_loc + file_name,sheetname=sheet)
df = db.download_file(file_loc,file_name,sheetname=sheet)

df.columns = ['TRR_Report_ID','Officer_Last_Name','Officer_First_Name','Star']

final_df = final_df.append(df)
final_df.reset_index(drop=True,inplace=True)
metadata_df = metadata_df.append(metadata_dataset(df,sheet))
metadata_df.reset_index(drop=True,inplace=True)

In [ ]:
out_path = local_path+'/output/TRRs/'
saving_files = "Star_No"

final_df.to_csv(out_path+saving_files+".csv",index=False)
final_df.to_excel(out_path+saving_files+".xlsx",index=False)

metadata_df.to_csv(out_path+saving_files+"_metadata.csv",index=False)

In [ ]:
sheet = [sheet for sheet in sheets if sheet=='ActionsResponses'][0]

final_df = pd.DataFrame()
metadata_df = pd.DataFrame()

##df = pd.read_excel(file_loc + file_name,sheetname=sheet)
df = db.download_file(file_loc,file_name,sheetname=sheet)

df.columns = ['TRR_Report_ID','Person','Resistance_Type','Action','Other_Description','Event','Force_Type'
              ,'Force_Type_New','Other_Force','Other_Force_Counter','Force_Type_3','Officer_Race']

## dropping fields that were joined manually
df = df[['TRR_Report_ID','Person','Resistance_Type','Action','Other_Description']]

final_df = final_df.append(df)
final_df.reset_index(drop=True,inplace=True)
metadata_df = metadata_df.append(metadata_dataset(df,sheet))
metadata_df.reset_index(drop=True,inplace=True)

In [ ]:
out_path = local_path+'/output/TRRs/'
saving_files = "Actions_Responses"

final_df.to_csv(out_path+saving_files+".csv",index=False)
final_df.to_excel(out_path+saving_files+".xlsx",index=False)

metadata_df.to_csv(out_path+saving_files+"_metadata.csv",index=False)

In [ ]:
sheet = [sheet for sheet in sheets if sheet=='WeaponDischarges'][0]

final_df = pd.DataFrame()
metadata_df = pd.DataFrame()

##df = pd.read_excel(file_loc + file_name,sheetname=sheet)
df = db.download_file(file_loc,file_name,sheetname=sheet)

df.columns = ['TRR_Report_ID','Weapon_Type','Weapon_Type_Description','Firearm_Make','Firearm_Model',
              'Firearm_Barrel_Length','Firearm_Caliber','Total_No_Shots','Firearm_Reloaded',
             'No_Cartridges_Reloaded','Handgun_Worn_Type','Handgun_Drawn_Type','Method_Used_Reload'
             ,'Sight_Used','Protective_Cover_Used','Discharge_Distance','Object_Struck_Discharge'
             ,'Discharge_Position','Type','Taser','Person_Shot','Event']

## dropping fields that were joined manually
df = df[['TRR_Report_ID','Weapon_Type','Weapon_Type_Description','Firearm_Make','Firearm_Model',
              'Firearm_Barrel_Length','Firearm_Caliber','Total_No_Shots','Firearm_Reloaded',
             'No_Cartridges_Reloaded','Handgun_Worn_Type','Handgun_Drawn_Type','Method_Used_Reload'
             ,'Sight_Used','Protective_Cover_Used','Discharge_Distance','Object_Struck_Discharge'
             ,'Discharge_Position']]

final_df = final_df.append(df)
final_df.reset_index(drop=True,inplace=True)
metadata_df = metadata_df.append(metadata_dataset(df,sheet))
metadata_df.reset_index(drop=True,inplace=True)

In [ ]:
out_path = local_path+'/output/TRRs/'
saving_files = "Weapon_Discharges"

final_df.to_csv(out_path+saving_files+".csv",index=False)
final_df.to_excel(out_path+saving_files+".xlsx",index=False)

metadata_df.to_csv(out_path+saving_files+"_metadata.csv",index=False)

In [ ]:
sheet = [sheet for sheet in sheets if sheet=='Charges'][0]

final_df = pd.DataFrame()
metadata_df = pd.DataFrame()

##df = pd.read_excel(file_loc + file_name,sheetname=sheet)
df = db.download_file(file_loc,file_name,sheetname=sheet)

df.columns = ['TRR_Report_ID','RD_No','Statute','Description']

final_df = final_df.append(df)
final_df.reset_index(drop=True,inplace=True)
metadata_df = metadata_df.append(metadata_dataset(df,sheet))
metadata_df.reset_index(drop=True,inplace=True)

In [ ]:
out_path = local_path+'/output/TRRs/'
saving_files = "Charges"

final_df.to_csv(out_path+saving_files+".csv",index=False)
final_df.to_excel(out_path+saving_files+".xlsx",index=False)

metadata_df.to_csv(out_path+saving_files+"_metadata.csv",index=False)

In [ ]:
sheet = [sheet for sheet in sheets if sheet=='SubjectWeapons'][0]

final_df = pd.DataFrame()
metadata_df = pd.DataFrame()

##df = pd.read_excel(file_loc + file_name,sheetname=sheet)
df = db.download_file(file_loc,file_name,sheetname=sheet)

df.columns = ['TRR_Report_ID','Weapon_Type','Firearm_Caliber','Weapon_Description','Event','Type','Type2']

## dropping fields that were joined manually
df = df[['TRR_Report_ID','Weapon_Type','Firearm_Caliber','Weapon_Description']]

final_df = final_df.append(df)
final_df.reset_index(drop=True,inplace=True)
metadata_df = metadata_df.append(metadata_dataset(df,sheet))
metadata_df.reset_index(drop=True,inplace=True)

In [ ]:
out_path = local_path+'/output/TRRs/'
saving_files = "Subject_Weapons"

final_df.to_csv(out_path+saving_files+".csv",index=False)
final_df.to_excel(out_path+saving_files+".xlsx",index=False)

metadata_df.to_csv(out_path+saving_files+"_metadata.csv",index=False)

In [ ]:
sheet = [sheet for sheet in sheets if sheet=='Statuses_OtherMembers'][0]

final_df = pd.DataFrame()
metadata_df = pd.DataFrame()

##df = pd.read_excel(file_loc + file_name,sheetname=sheet)
df = db.download_file(file_loc,file_name,sheetname=sheet)

df.columns = ['TRR_Report_ID','Status','Stat_Date','Officer_First_Name','Officer_Last_Name','Star','Rank'
             ,'Officer_Gender','Officer_Race','Officer_Age','Appointed_Date']

final_df = final_df.append(df)
final_df.reset_index(drop=True,inplace=True)
metadata_df = metadata_df.append(metadata_dataset(df,sheet))
metadata_df.reset_index(drop=True,inplace=True)

In [ ]:
out_path = local_path+'/output/TRRs/'
saving_files = "Statuses_Other_Members"

final_df.to_csv(out_path+saving_files+".csv",index=False)
final_df.to_excel(out_path+saving_files+".xlsx",index=False)

metadata_df.to_csv(out_path+saving_files+"_metadata.csv",index=False)